In [15]:

import math
f = open("input.txt", "r")
l = f.read().split("\n")[:-1]

ll = [[ord(y) for y in x] for x in l]


In [16]:
start = ord("S")
end = ord("E")
dimX = len(ll[0])
dimY = len(ll)
target = (0,0)
startPoint = (0,0)



In [17]:
for y in range(0,len(ll)):
    for x in range(0,len(ll[0])):
        if ll[y][x] == end:
            ll[y][x] = ord("z")
            target = (x,y)
        if ll[y][x] == start:
            startPoint = (x,y)
            ll[y][x] = ord("a")


In [18]:
from copy import copy
class MyNode():
    def __init__(self,x,y,g,pred,tar):
        self.pt =(x,y)
        self.pred = pred
        self.suc = None
        self.h = self.getDist(tar)
        self.g = g
    def getCost(self):
        return self.h + self.g
    def getDist(self,tar):
        return math.sqrt(math.pow((tar[0] - self.pt[0]) + (tar[1] - self.pt[1]),2))
    def ptAsString(self):
        return str(self.pt[0]) + "-" + str(self.pt[1] )
    def compare(self,other):
        if(other[0] == self.pt[0] and other[1] == self.pt[1]):
            return True
        return False
    def __copy__(self):
        cls = self.__class__
        result = cls.__new__(cls)
        result.__dict__.update(self.__dict__)
        return result

In [19]:
def getMin(nodeLst):
    idx = 0
    for i in range(0,len(nodeLst)):
        if nodeLst[i].getCost() < nodeLst[idx].getCost():
            idx = i
    return idx


def getSuc(elem,tar,nodeList,dimX,dimY):
    ret = []

    for i in [-1,0,1]:
        for n in [-1,0,1]:
            x = elem.pt[0]
            y = elem.pt[1]
            
            _x = x +n
            _y = y +i
            
            if _x >= dimX or _y >= dimY: continue
            
            if _x <0 or _y <0: continue
            if abs(n+i) != 1: continue
            try:
                if nodeList[y][x]+1 < nodeList[_y][_x]: continue
            except:
                print("ERRRR","NEWX",_x,"NEWY",_y,"X",x,"Y","DIMX",dimX,"DIMY",dimY)
            #print("CHECK",_x,_y)
            tt = MyNode(_x,_y,elem.g+nodeList[_y][_x],None,tar)
            
            ret.append(tt)
            #print(ret)

    return ret


def elemInList(elem,nodeList):
    for i in range(0,len(nodeList)):
        if nodeList[i].compare(elem.pt):
            #print(nodeList[i].pt, elem.pt)
            return True
    return False

def checkBetter(elem,nodeList):
    for e in nodeList:
        if elem.compare(e.pt) and elem.getCost() > e.getCost():
            return True
    return False


def insertOrUpdate(elem,nodeList):
    for i in range(0,len(nodeList)):
        if nodeList[i].compare(elem.pt):
            if elem.g < nodeList[i].g:
                nodeList[i] = elem
            return

    nodeList.append(elem)



def expand(curr,openList,closedList,tar,arr,dimX,dimY):
 
    suc = getSuc(curr,tar,arr,dimX,dimY)
    for e in suc:
        
        
        if(elemInList(e,closedList)): continue
        if(checkBetter(e,openList)): continue
        
        e.pred = curr

        insertOrUpdate(e,openList)

In [20]:
def astar(nodeList, strt,tar,dimX,dimY):
    
    openList=[]
    closedList = []
                   
    start = MyNode(strt[1],strt[0],0,None,tar)
    openList.append(start)
    while(len(openList)>0):
        
        min = getMin(openList)
        elem = openList[min]
        #print(elem.pt)
        del openList[min]
        if elem.compare(tar):
            return elem
        closedList.append(elem)
        expand(elem,openList,closedList,tar,nodeList,dimX,dimY)

In [21]:
from collections import defaultdict
from ipywidgets import IntProgress
from IPython.display import display
rList = []
nnList = []
dct = defaultdict(int)
checkLst = []
pBar  = IntProgress(min=0, max=(dimY-1)*(dimX-1)) # instantiate the bar
display(pBar)

for xx in range(0,dimY-1):
    for yy in range(0,dimY):
        pBar.value+=1

        if ll[yy][xx] == ord("a"):
            startPoint = (yy,xx)
            #print("start", startPoint)
            fromDct = False
            ppath = None
            lastElem =None
            path = None
            if dct[str(xx) +"-"+str(yy)] != 0:
                ppath = nnList[dct[str(xx) +"-"+str(yy)]-1][0]
                cur= nnList[dct[str(xx) +"-"+str(yy)]-1][1]
                fromDct = True
            else:
                path = astar(ll,startPoint,target,dimX,dimY)
                lastElem = path
                cur = path

            if path == None and not fromDct:
                continue
            
            ct = 0
      
            while cur.pred != None  :
                ct+=1
                if not fromDct:
                    nnList.append((cur,lastElem))
                    #defaultdict gets a 0 entry for every query so 0 (in this case)
                    #should not be used.
                    dct[cur.ptAsString()]  = len(nnList)
                
                cur = cur.pred
                if fromDct and cur == ppath:
                    break
            if rList != []:
                if ct < min(rList):
                    print("New min", ct)
            rList.append(ct)
  

print(min(rList))

IntProgress(value=0, max=6480)

KeyboardInterrupt: 